In [1]:
!pip install datasets
!pip install transformers
!pip install rouge_score
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

# Mapping Dzongkha text to Roman Text

The Dzongkha language is notoriously unphonetic, creating a particular challenge for those seeking to learn the language. This project is an attempt to map Dzongkha characters to their romanized equivalents.

The data for this project is notably meager, so the results will likely not be ideal. Nonetheless, this is a good exercise in romanization pipelines, which are particularly useful for projects such as automatic speech recognition. Indeed, some semblance of linguistic justice in AI tools requires the interoperability of various texts. 

# Preprocess Data

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [28]:
df = pd.read_csv('/content/thai_romanization.csv',delimiter='	', nrows = 20000).dropna(axis=0)

In [29]:
len(df)

19997

In [30]:
train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])

In [31]:
print(len(train),len(test),len(validate))

11998 4000 3999


In [32]:
train.to_csv('thai_ro_train.csv')
validate.to_csv('thai_ro_validate.csv')
test.to_csv('thai_ro_test.csv')

Load to Hugging Face Dataset Format

In [33]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': '/content/thai_ro_train.csv', 'test': '/content/thai_ro_test.csv', 'validation':'/content/thai_ro_validate.csv'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1996b16f3b7adcab/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
dataset['train'][221]

{'Unnamed: 0': 13591, 'word': 'บ้านหนองตับ', 'romanization': 'bannongtap'}

In [12]:
from transformers import T5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer

# Fine-Tuning with Huggingface



In [13]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# !apt install git-lfs
# !git config --global credential.helper store

In [14]:
import transformers

print(transformers.__version__)

4.26.1


In [15]:
from datasets import load_metric

metric = load_metric("rouge")

<ipython-input-15-1a0e7328e82b>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [16]:
model_checkpoint = 'google/byt5-small'

In [17]:
from transformers import T5ForConditionalGeneration, AutoTokenizer


# model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [21]:
tokenizer("บ้านหนองเลา")


{'input_ids': [227, 187, 157, 227, 188, 140, 227, 187, 181, 227, 187, 156, 227, 187, 174, 227, 187, 156, 227, 187, 176, 227, 187, 138, 227, 188, 131, 227, 187, 168, 227, 187, 181, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
max_input_length = 512
max_target_length = 128

def preprocess(examples):
    inputs = [doc for doc in examples["word"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["romanization"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [23]:
preprocess(dataset['train'][4:6])

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[227, 187, 176, 227, 187, 183, 227, 187, 142, 227, 187, 179, 227, 187, 156, 227, 187, 179, 227, 187, 164, 227, 187, 183, 1], [227, 187, 159, 227, 187, 181, 227, 187, 173, 227, 187, 187, 227, 187, 132, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[108, 118, 100, 113, 100, 112, 108, 1], [115, 107, 100, 118, 120, 110, 1]]}

In [34]:
tokenized_data = dataset.map(preprocess, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [35]:
tokenized_data['train']['labels'][0]

[117,
 114,
 110,
 119,
 107,
 100,
 113,
 106,
 117,
 100,
 101,
 114,
 115,
 115,
 117,
 100,
 118,
 100,
 119,
 1]

In [36]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [37]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 10 for key, value in result.items()}
    ## usually multiply by 100 instead of 10, but Rogue may not be the best metric, so we're weakening it's effect
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [46]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetune-thai-to-romanized",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=8,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

## Reduced batch size from 16 to 8 to address memory issue
## Changed fp16 to false because of conversion problems

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [43]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [40]:
# import torch
# torch.cuda.empty_cache()

In [47]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/byt5-small-finetune-thai-to-romanized is already a clone of https://huggingface.co/caffsean/byt5-small-finetune-thai-to-romanized. Make sure you pull the latest changes with `repo.git_pull()`.


In [48]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.896200,1.641868,0.034800,0.000000,0.035100,0.034500,11.676700
2,1.186700,0.648670,1.473900,0.046300,1.479300,1.480400,11.617900
3,0.610800,0.264848,4.943400,0.233800,4.937300,4.941300,11.637900
4,0.363000,0.156136,6.889800,0.364000,6.882700,6.893500,11.808000
5,0.267100,0.113197,7.557800,0.485500,7.549600,7.560400,11.850700
6,0.219800,0.097599,7.787600,0.511000,7.781900,7.786400,11.877200
7,0.182300,0.091987,7.883000,0.542200,7.879700,7.883400,11.870700
8,0.185400,0.088073,7.922300,0.551600,7.917600,7.920500,11.877500


Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_

TrainOutput(global_step=12000, training_loss=0.7239941743214925, metrics={'train_runtime': 7247.4972, 'train_samples_per_second': 13.244, 'train_steps_per_second': 1.656, 'total_flos': 9493141505533440.0, 'train_loss': 0.7239941743214925, 'epoch': 8.0})

In [49]:
trainer.push_to_hub()

Saving model checkpoint to byt5-small-finetune-thai-to-romanized
Configuration saved in byt5-small-finetune-thai-to-romanized/config.json
Configuration saved in byt5-small-finetune-thai-to-romanized/generation_config.json
Model weights saved in byt5-small-finetune-thai-to-romanized/pytorch_model.bin
tokenizer config file saved in byt5-small-finetune-thai-to-romanized/tokenizer_config.json
Special tokens file saved in byt5-small-finetune-thai-to-romanized/special_tokens_map.json


Upload file runs/Feb20_13-37-41_d7c52de7b925/events.out.tfevents.1676900277.d7c52de7b925.403.2: 100%|#########…

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/caffsean/byt5-small-finetune-thai-to-romanized
   47cefaa..d8dc748  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/caffsean/byt5-small-finetune-thai-to-romanized
   47cefaa..d8dc748  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 7.9223}]}
To https://huggingface.co/caffsean/byt5-small-finetune-thai-to-romanized
   d8dc748..34faab9  main -> main

   d8dc748..34faab9  main -> main



'https://huggingface.co/caffsean/byt5-small-finetune-thai-to-romanized/commit/d8dc748eac969706f715a191c97fbd2e23e2ce22'

In [50]:
from transformers import T5ForConditionalGeneration, AutoModelForSeq2SeqLM

custom_model = "caffsean/byt5-small-finetune-thai-to-romanized"

In [51]:
model = T5ForConditionalGeneration.from_pretrained(custom_model)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--caffsean--byt5-small-finetune-thai-to-romanized/snapshots/34faab93e9619162f911796120d94a02c08bb792/config.json
Model config T5Config {
  "_name_or_path": "google/byt5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3584,
  "d_kv": 64,
  "d_model": 1472,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 4,
  "num_heads": 6,
  "num_layers": 12,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "ByT5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "use_cache

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--caffsean--byt5-small-finetune-thai-to-romanized/snapshots/34faab93e9619162f911796120d94a02c08bb792/pytorch_model.bin
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at caffsean/byt5-small-finetune-thai-to-romanized.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.


loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--caffsean--byt5-small-finetune-thai-to-romanized/snapshots/34faab93e9619162f911796120d94a02c08bb792/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



In [52]:

test = df.sample(10)


In [53]:
th_list = list(test['word'].values)
ro_list = list(test['romanization'].values)

In [57]:
def generate_samples_from_list(word_list, model, tokenizer, min_length=20):
    generated_text = []
    for word in tqdm(word_list):
        encoded_input = tokenizer(word)
        with torch.no_grad():
              generated_ids = model.generate(
                    input_ids = torch.LongTensor(encoded_input['input_ids']).unsqueeze(0),
                    attention_mask = torch.LongTensor(encoded_input['attention_mask']).unsqueeze(0), 
                    min_length=min_length, 
                    num_beams=5,
                    repetition_penalty=10.0, 
                    length_penalty=1.0, 
                    early_stopping=True
                    )
        text = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
      
        generated_text.append([word,text[0].split('\n')[0]])
      
    return pd.DataFrame(generated_text)

In [ ]:
sample = generate_samples_from_list(th_list, model, tokenizer)

In [60]:
from nltk.metrics import edit_distance

In [67]:
sample[2] = ro_list


def edit_distance(df):
  distances = []
  for idx in range(len(df)):
    ed = nltk.edit_distance(df.iloc[idx][1],df.iloc[idx][2])
    distances.append(ed)

  df['edit_distance'] = distances
  return df

edit_distance(sample)


,0,1,2,edit_distance
0,โรคทางระบบประสาท,rokthangrabopprasat,rokthangrabopprasat,0
1,พีฟลอกซาซิน,phifloksasinพin,phifloksasin,3
2,เกาะคริสต์มาส,kokhritsamatاṌch,kokhritmat,6
3,โกวิท,kawithorotchoetभp,kowit,13
4,บ้านทุ่งครุ,banthungkhru,banthungkhru,0
5,บ้านบ่อแพ,banbophae,banbophae,0
6,เหมาะเจาะ,maochorae,mocho,4
7,ต้นไทรหิน,tonsaihinะnṣuen,tonsaihin,6
8,การหาเสียงเลือกตั้ง,kanhasianglueaktang,kanhasianglueaktang,0
9,พะเนิน,phanoenเenḣuenE,phanoen,8
